In [ ]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt 
import plotly.express as px
%matplotlib inline
import warnings
import numpy as np

# Suppress FutureWarning messages
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv("Resources/crypto_market_data.csv")

# Strip any whitespace in column names
df_market_data.columns = df_market_data.columns.str.strip()

# Set index column to 'coin_id'
df_market_data.set_index("coin_id", inplace=True)

# Display sample data
df_market_data.tail(10)

In [ ]:
# Check Dataframe index
print(df_market_data.index)

In [ ]:
num_rows, num_columns = df_market_data.shape

print("Number of rows:", num_rows)
print("Number of columns:", num_columns)

In [ ]:
# Generate summary statistics
df_market_data.describe()

In [ ]:
# Check the DataFrame data types
df_market_data.dtypes

In [ ]:
df_market_data.isnull().sum() 

In [ ]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90)

---

### Prepare the Data

In [ ]:
print(df_market_data.columns) 

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file. Scale all columns with numerical values.
#Note: normalization using StandardScaler() transforms selected features into NumPy array

market_data_scaled = StandardScaler().fit_transform(df_market_data[["price_change_percentage_24h","price_change_percentage_7d","price_change_percentage_14d","price_change_percentage_30d","price_change_percentage_60d","price_change_percentage_200d","price_change_percentage_1y"]])

# Diplay the first five rows of the scaled data
market_data_scaled[0:5]

In [ ]:
# Create a DataFrame with the scaled data
df_market_data_scaled = pd.DataFrame(
    market_data_scaled,
    columns=["price_change_percentage_24h","price_change_percentage_7d","price_change_percentage_14d","price_change_percentage_30d","price_change_percentage_60d","price_change_percentage_200d","price_change_percentage_1y"])

In [ ]:
if type(df_market_data_scaled) == pd.DataFrame:
    print("df_market_data_scaled is a DataFrame")
else:
    print("df_market_data_scaled is not a DataFrame")

In [ ]:
# Copy the crypto names from the original data
#i.e., Create a 'coin_id' column in the df_market_data_scaled DataFrame using the index of the original df_market_data DataFrame
df_market_data_scaled["coin_id"] = df_market_data.index

# Set the coinid column as index
df_market_data_scaled = df_market_data_scaled.set_index("coin_id")

# Display sample data
df_market_data_scaled.head()

---

### Find the Best Value for k Using the Original Data.

In [ ]:
# Create a list with the number of k-values from 1 to 11

k = list(range(1, 12))  

print(k)

In [ ]:
# Create an empty list to store the inertia values
inertia =[]

OMP_NUM_THREADS=1

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list

for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(df_market_data_scaled)
    inertia.append(k_model.inertia_)

print(inertia)

In [ ]:
OMP_NUM_THREADS=1

# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)

print(df_elbow)

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.

df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** Four is the best value for k. 

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [ ]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=1)

In [ ]:
if type(df_market_data_scaled) == pd.DataFrame:
    print("df_market_data_scaled is a DataFrame")
else:
    print("df_market_data_scaled is not a DataFrame")

In [ ]:
# Fit the K-Means model using the scaled data
model.fit(df_market_data_scaled)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled data
k_4 = model.predict(df_market_data_scaled)

# Print the predictions
print(k_4)


In [ ]:
# Print the resulting array of cluster values.
# Use enumerate to iterate over array k_4 and retrieve row index and corresponding cluster value
for index, cluster_value in enumerate(k_4):
    print(f"Row index: {index}, Cluster value: {cluster_value}")

In [ ]:
# Create a copy of the DataFrame
df_market_data_scaled_predictions=df_market_data_scaled.copy()

In [ ]:
# Check the length of k_4
print("Length of k_4:", len(k_4))

# Check the first few values of k_4
print("First few values of k_4:", k_4[:5])

In [ ]:
# Check the type of df_market_data_scaled_predictions
print("Type of df_market_data_scaled_predictions:", type(df_market_data_scaled_predictions))

# If df_market_data_scaled_predictions is a NumPy array, convert it to a DataFrame
if isinstance(df_market_data_scaled_predictions, np.ndarray):
    df_market_data_scaled_predictions = pd.DataFrame(df_market_data_scaled_predictions)

# Verify the column names in df_market_data_scaled_predictions
print("Column names in df_market_data_scaled_predictions:", df_market_data_scaled_predictions.columns)

In [ ]:
# Add a new column to the DataFrame with the predicted clusters
df_market_data_scaled_predictions['crypto_cluster_4']=k_4

# Display sample data
df_market_data_scaled_predictions.head()

In [ ]:
#assistance with syntax provided by chat gpt

counts_by_unique_value = df_market_data_scaled_predictions['crypto_cluster_4'].value_counts()
print(counts_by_unique_value)

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.

# Create new DataFrame with index included as column
df_market_data_scaled_predictions_with_index = df_market_data_scaled_predictions.reset_index()

# Create the scatter plot with hover data including 'coin_id'
fig = px.scatter(df_market_data_scaled_predictions_with_index, 
                 x='price_change_percentage_24h', 
                 y='price_change_percentage_7d', 
                 color='crypto_cluster_4', 
                 hover_data=['coin_id', 'crypto_cluster_4'], 
                 color_continuous_scale='viridis')

# Update layout
fig.update_layout(title='Scatter Plot of Cluster Assignments with Scaled Data', 
                  xaxis_title='price_change_percentage_24h', 
                  yaxis_title='price_change_percentage_7d', 
                  coloraxis_colorbar=dict(title='Cluster'))

# Show the plot
fig.show()

---

### Optimize Clusters with Principal Component Analysis.

In [ ]:
# Import the PCA module
from sklearn.decomposition import PCA

In [ ]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [ ]:
# Use the PCA model with `fit_transform` to reduce to three principal components.
market_data_scaled_pca = pca.fit_transform(df_market_data_scaled)

if type(market_data_scaled_pca) == pd.DataFrame:
    print("market_data_scaled_pca is a DataFrame")
else:
    print("market_data_scaled_pca is not a DataFrame")

In [ ]:
# Create a new DataFrame with the PCA data
df_market_data_scaled_pca = pd.DataFrame(market_data_scaled_pca)

# Assign new column names
df_market_data_scaled_pca.columns = ['PC1', 'PC2', 'PC3'] 

# View the first five rows of the DataFrame. 
df_market_data_scaled_pca[:5]

In [ ]:
num_rows, num_columns = df_market_data_scaled_pca.shape

print("Number of rows:", num_rows)
print("Number of columns:", num_columns)

In [ ]:
# Retrieve the explained variance to determine how much information can be attributed to each principal component.

# Calculate the PCA explained variance ratio
pca.explained_variance_ratio_

# total explained variance
total_explained_variance = sum(pca.explained_variance_ratio_)

print(pca.explained_variance_ratio_)
print("Total explained variance:", total_explained_variance)

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** PC1 explains 37%, PC2 explains 35%, and PC3 explains 18% of the variance. The total explained variance is 89.5%.

In [ ]:
print(df_market_data)

In [ ]:
# Copy the crypto names from the original data
#i.e., Create a 'coin_id' column in the df_market_data_scaled_pca DataFrame using the index of the original df_market_data DataFrame
df_market_data_scaled_pca["coin_id"] = df_market_data.index

# Set the coinid column as index
df_market_data_scaled_pca = df_market_data_scaled_pca.set_index("coin_id")

# Display sample data
df_market_data_scaled_pca.head()

---

### Find the Best Value for k Using the PCA Data

In [ ]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 11))  

print(k)

In [ ]:
# Create an empty list to store the inertia values
inertia_pca= []

# Create a for loop to compute the inertia with each possible value of k
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(df_market_data_scaled_pca)
    inertia_pca.append(k_model.inertia_)

# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca` (i.e., pca_df)
# 3. Append the model.inertia_ to the inertia list

print(inertia_pca)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
pca_elbow_data = {"k": k, "inertia": inertia_pca}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow_pca = pd.DataFrame(pca_elbow_data)

print(df_elbow_pca)

In [ ]:
# Plot a line chart with all the inertia values computed with the different values of k to visually identify the optimal value for k.

df_elbow_pca.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** The best value for 'k' is still 4, based on the new elbow curve. 


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** It does not differ from the best k value found using the original data. 

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [ ]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=1)

In [ ]:
# Fit the K-Means model using the PCA data
model.fit(df_market_data_scaled_pca)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the PCA data
k_4 = model.predict(df_market_data_scaled_pca)

# Print the resulting array of cluster values.
print(k_4)

In [ ]:
for index, cluster_value in enumerate(k_4):
    print(f"Row index: {index}, Cluster value: {cluster_value}")

In [ ]:
if type(df_market_data_scaled_pca) == pd.DataFrame:
    print("df_market_data_scaled_pca is a DataFrame")
else:
    print("df_market_data_scaled_pca is not a DataFrame")

In [ ]:
# Create a copy of the DataFrame with the PCA data
df_market_data_scaled_pca_predict = pd.DataFrame(df_market_data_scaled_pca).copy()

# Add a new column to the DataFrame with the predicted clusters
df_market_data_scaled_pca_predict['crypto_cluster_4']=k_4

df_market_data_scaled_pca_predict.columns = ['PC1', 'PC2', 'PC3', 'crypto_cluster_4'] 

# Display sample data
df_market_data_scaled_pca_predict.head()

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify the cryptocurrency represented by each data point.

# Create new DataFrame with index included as column
df_market_data_scaled_pca_predict_with_index = df_market_data_scaled_pca_predict.reset_index()

# Create the scatter plot with hover data including 'coin_id'
fig = px.scatter(df_market_data_scaled_pca_predict_with_index, 
                 x='PC1', 
                 y='PC2', 
                 color='crypto_cluster_4', 
                 hover_data=['coin_id', 'crypto_cluster_4'], 
                 color_continuous_scale='viridis')

# Update layout
fig.update_layout(title='Scatter Plot of Cluster Assignments with Scaled Data and PCA', 
                  xaxis_title='PC1', 
                  yaxis_title='PC2', 
                  coloraxis_colorbar=dict(title='Cluster'))

# Show the plot
fig.show()

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [ ]:
# Composite plot to contrast the Elbow curves
#ChatGPT assisted with how to plot two curves side by side

import matplotlib.pyplot as plt

# df_elbow and df_elbow_pca contain inertia values for two data sets
# Create new figure and set of subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 3))

# Plot first Elbow curve
ax1.plot(df_elbow['k'], df_elbow['inertia'], marker='o', linestyle='-', label='Original Data')
ax1.set_title('Elbow Curve for Original Data')
ax1.set_xlabel('Number of Clusters (k)')
ax1.set_ylabel('Inertia')
ax1.legend()

# Plot second Elbow curve
ax2.plot(df_elbow_pca['k'], df_elbow_pca['inertia'], marker='s', linestyle='--', label='PCA Data')
ax2.set_title('Elbow Curve for PCA Data')
ax2.set_xlabel('Number of Clusters (k)')
ax2.set_ylabel('Inertia')
ax2.legend()

# Adjust layout to prevent overlap
plt.tight_layout()

# Display composite plot
plt.show()

In [ ]:
print(df_market_data_scaled_predictions.columns)
print(df_market_data_scaled_pca_predict.columns)

In [ ]:
# Composite plot to contrast the clusters
#ChatGPT assisted with how to plot two curves side by side and create hover data

import plotly.express as px
from plotly.subplots import make_subplots

# Create the scatter plot with hover data including 'coin_id'
fig1 = px.scatter(df_market_data_scaled_predictions_with_index, 
                 x='price_change_percentage_24h', 
                 y='price_change_percentage_7d', 
                 color='crypto_cluster_4', 
                 hover_data=['coin_id', 'crypto_cluster_4'], 
                 color_continuous_scale='viridis')

# Update layout
fig1.update_layout(title='Scatter Plot of Cluster Assignments with Scaled Data', 
                  xaxis_title='price_change_percentage_24h', 
                  yaxis_title='price_change_percentage_7d', 
                  coloraxis_colorbar=dict(title='Cluster'))

# Create the scatter plot with hover data including 'coin_id'
fig2 = px.scatter(df_market_data_scaled_pca_predict_with_index, 
                 x='PC1', 
                 y='PC2', 
                 color='crypto_cluster_4', 
                 hover_data=['coin_id', 'crypto_cluster_4'], 
                 color_continuous_scale='viridis')

# Update layout
fig2.update_layout(title='Scatter Plot of Cluster Assignments with Scaled Data and PCA', 
                  xaxis_title='PC1', 
                  yaxis_title='PC2', 
                  coloraxis_colorbar=dict(title='Cluster'))

# Combine the two plots into a side-by-side layout
combined_fig = make_subplots(rows=1, cols=2, subplot_titles=('Cluster Chart 1: Scaled Data', 'Cluster Chart 2: Scaled Data with PCA'))

# Add traces to the subplots
combined_fig.add_trace(fig1.data[0], row=1, col=1)
combined_fig.add_trace(fig2.data[0], row=1, col=2)

# Update x-axis and y-axis labels for the first subplot
combined_fig.update_xaxes(title_text="price_change_percentage_24h", row=1, col=1)
combined_fig.update_yaxes(title_text="price_change_percentage_7d", row=1, col=1)

# Update x-axis and y-axis labels for the second subplot
combined_fig.update_xaxes(title_text="PC1", row=1, col=2)
combined_fig.update_yaxes(title_text="PC2", row=1, col=2)

# Update layout and show the combined plot
combined_fig.update_layout(height=600, width=1000, showlegend=True, hovermode='closest', title="Combined Plots with Axes Labels")
combined_fig.show()

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** When using fewer features, the four clusters are more defined than previously. In Cluster Chart 1, with the scaled data, the cluster zero plot looks like it could easily be a cluster 1 plot. The cluster 3 point looks like an outlier compared to the rest of the plots in the graph. In Cluster Chart 2, with the scaled data and PCA, cluster 2 and 0 are more tightly clustered but show some overlap. The cluster 3 plot again appears to be an outlier, although it's position on the graph has changed dramatically. In this second chart, the cluster 1 plot is now an outlier. It no longer "sits" with cluster 0. 